In [3]:
# Import libraries
import psycopg2 as ps

import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')
import numpy as np

from Preprocessing_Functions import *
from Classification_Metrics_Functions import *

from pickle import dump 
from pickle import load

In [4]:
# Connect to database and get nhl_view
try:
    conn = ps.connect('''host=localhost
                         dbname=nhldb
                         user=postgres
                         password=iEchu133
                         ''')
    cur = conn.cursor()

    print('Successfully connected to nhldb.')

except ps.Error as e:
    print('\n Database Error:')
    print(e)
    
query = '''SELECT *
           FROM nhl_view;'''

cur.execute(query)
nhl_df = cur.fetchall()

nhl_df_columns = [
    'team',
    'season_year_range',
    'games_played',
    'wins',
    'losses',
    'ot',
    'pts',
    'pt_pctg',
    'goals_per_game',
    'goals_against_per_game',
    'evgga_ratio',
    'power_play_percentage',
    'power_play_goals',
    'power_play_goals_against',
    'power_play_opportunities',
    'penalty_kill_percentage',
    'shots_per_game',
    'shots_allowed',
    'win_score_first',
    'win_opp_score_first',
    'win_lead_first_per',
    'win_lead_second_per',
    'win_outshoot_opp',
    'win_outshot_by_opp',
    'face_offs_taken',
    'face_offs_won',
    'face_offs_lost',
    'face_off_win_percentage',
    'shooting_pctg',
    'save_pctg',
    'failed_shots_per_game',
    'saves_per_game',
    'ties',
    'power_play_efficiency',
    'made_playoffs',
    'adjWins'
]

nhl_df = pd.DataFrame( nhl_df, columns = nhl_df_columns)

conn.close()

# Claculate and remove adjusted wins outliers
outliers = remove_outliers(df = nhl_df, col = 'adjWins')
nhl_df = nhl_df.loc[(nhl_df['adjWins'] > outliers[1]) & (nhl_df['adjWins'] < outliers[3])].reset_index(drop = True)

remove_features = [
    'team', 
    'season_year_range', 
    'games_played',
    'wins', 
    'losses', 
    'ot',
    'pts', 
    'pt_pctg',
    'goals_per_game',
    'goals_against_per_game',
    'evgga_ratio', 
    'power_play_percentage',
    'power_play_goals',
    'power_play_goals_against',
    'power_play_opportunities',
    'shots_per_game', 
    'shots_allowed',
    'win_score_first',
    'win_opp_score_first', 
    'win_lead_first_per',
    'win_lead_second_per', 
    'win_outshoot_opp', 
    'win_outshot_by_opp',
    'face_offs_taken', 
    'face_offs_won',
    'face_offs_lost',
    'face_off_win_percentage', 
    'ties',
    'power_play_efficiency', 
    'adjWins'
]

# Remove unneeded features
nhl_df = nhl_df.drop(
    remove_features, 
    axis = 1, 
    errors = 'ignore'
)

# Multiply savePctg by 100 to get percent
nhl_df['save_pctg'] = nhl_df['save_pctg'] * 100

nhl_df.info()
nhl_df.head()

Successfully connected to nhldb.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 922 entries, 0 to 921
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   penalty_kill_percentage  922 non-null    float64
 1   shooting_pctg            922 non-null    float64
 2   save_pctg                922 non-null    float64
 3   failed_shots_per_game    922 non-null    float64
 4   saves_per_game           922 non-null    float64
 5   made_playoffs            922 non-null    int64  
dtypes: float64(5), int64(1)
memory usage: 43.3 KB


,penalty_kill_percentage,shooting_pctg,save_pctg,failed_shots_per_game,saves_per_game,made_playoffs
0,75.3,10.9,86.3,23.511263,27.616000,0
1,81.5,14.7,89.9,25.824575,30.015363,1
2,78.1,12.3,88.0,27.965338,27.764000,1
3,76.6,11.9,87.3,32.376750,25.011450,1
4,76.0,11.2,86.6,25.163700,31.479100,0
